# Leia V3

This version of Leia uses GPT-4 in combination with the function tool to Lookup LeiaActions and Controls.

In [ ]:
import os
import json
import sys

## Read and set API keys

In [ ]:
# Open and read the config file
with open('config.json', 'r') as config_file:
    config_data = json.load(config_file)

# Retrieve the API key from the config data
api_key = config_data['api_key']
os.environ['OPENAI_API_KEY'] = api_key

## Logging

In [ ]:
# Set up logging
import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO) #DEBUG, INFO, WARNING, ERROR, CRITICAL
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import tiktoken
from llama_index.callbacks import CallbackManager, TokenCountingHandler
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

# you can set a tokenizer directly, or optionally let it default
# to the same tokenizer that was used previously for token counting
# NOTE: The tokenizer should be a function that takes in text and returns a list of tokens
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-4").encode,
    verbose=True  # set to true to see usage printed to the console
    )
callback_manager = CallbackManager([token_counter])

## Set up the functions to retrieve Actions and Controls

define a pydantic class for action output (This is not used at the moment at all because it worked right out of the box! in he future we might want to use this to define the output of the action and format it as a json for easier parsing.)

In [ ]:
from pydantic import BaseModel
from typing import List
from guidance.llms import OpenAI

class Arguments(BaseModel):
    name: str
    value: str #can we define more possibilities here?

class Action(BaseModel):
    name: str
    required_arguments: List[Arguments]
    optional_arguments: List[Arguments]



### Load the Actions index

In [ ]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage/actions")
# load index
actions_index = load_index_from_storage(storage_context)
actions_engine = actions_index.as_query_engine(similarity_top_k=3) #uses the default llm! Not gpt4!

### Load the Controls index

### Load the Documentation index

In [ ]:
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage/documentation")
# load index
docu_index = load_index_from_storage(storage_context)
docu_engine = docu_index.as_query_engine(similarity_top_k=5) #uses the default llm! Not gpt4!

## provide tool context

In [24]:
from llama_index.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=actions_engine,
        metadata=ToolMetadata(
            name="LeiaActions",
            description="provides documentation and descriptions for all implemented LeiaActions. try finding the appropriate action for a given user request and fill in the arguments. in case a required argument is not provided by context or user input, ask the user for it. LeiaActions are intended to be used under the hood of LiquidEarth, so the user should not be aware of them"
        ),
    ),
    QueryEngineTool(
        query_engine=docu_engine,
        metadata=ToolMetadata(
            name="Documentation",
            description="provides general information, user manual and the roadmap for LiquidEarth. pass the entire user request to the tool to find the appropriate documentation and use it for your response."
        ),
    )
]

## Test the engine

In [25]:
from llama_index.agent import OpenAIAgent
agent = OpenAIAgent.from_tools(query_engine_tools, verbose=True, system_prompt="You are Leia, the LiquidEarth Intelligent Assistant. You are helping a user with a question about LiquidEarth. You are very smart and friendly and always in a great mood. \n")

In [26]:
response = agent.chat("the background is very bright. can you set it to a dark grey?")
print(str(response))

=== Calling Function ===
Calling function: LeiaActions with args: {
  "input": "set background color to dark grey"
}
Got output: 
LeiaActionsSwitchBackgroundColor(color=#333333)
Retrying llama_index.llms.openai_utils.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..
Sure! I have set the background color to dark grey. Is there anything else I can help you with?


In [ ]:
agent.chat_repl()



## custom Agent (wip)

In [ ]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    system_prompt="You are Leia, the LiquidEarth Intelligent Assistant. You are helping a user with a question about LiquidEarth. You are very smart and friendly and always in a great mood. general information and documentation can be retrieved with the'Documentation' tool. To perform user requested actions in LiquidEarth, use the 'LeiaActions' Tool. \n"
)

In [ ]:
response = agent.chat("What is the difference between a space and a project?")
print(response)


In [ ]:
response = agent.chat("create a new space for me called 'test space'")
print(response)

## GPT 4

In [ ]:
from llama_index.llms import OpenAI
from llama_index import ServiceContext

gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4,callback_manager=callback_manager)

### create Prompt Template

In [ ]:
from llama_index import Prompt
# define custom Prompt
TEMPLATE_STR = (
    "You are Leia, the LiquidEarth Intelligent Assistant. You are helping a user with a question about LiquidEarth. You are very smart and friendly and always in a great mood.\n"
    "In LiquidEarth, a 'Space' and a 'Project are the same thing. We have provided Documentation on the software and further information below. In some cases the metadata includes a 'Control' Field that points to a UI element in the app associated to the described functionality. this is only for internal use. when describing controls to the user, use descriptions and names from the text, not the 'control' values. \n"
    "If the user asks you to do something in LiquidEarth, look for the right 'LeiaAction' in the context and compile the function call based on the information and example. do not use any Actions that are not documented. return the function call at the end of your response in curly braces. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Answer the question for a human to understand. Additionally, return the 'Control' properties in the order of operations in the following form at the end of your response: [[control1], [control2], ...]. Append the list to your response without further comment. If no controls are found, do not comment it. Never include any controls that are not specified in the Metadata Field in the provided documentation. Do not interpret any controls from the text body. If the answer requires multiple steps, describe each step in detail. Given this information, please answer the question: {query_str}\n"
)
QA_TEMPLATE = Prompt(TEMPLATE_STR)

In [ ]:
query_engine = index.as_query_engine(service_context=service_context_gpt4, text_qa_template=QA_TEMPLATE, retriever_mode="embedding",callback_manager=callback_manager)

In [ ]:
#dirty hack: trying to increase the context size
query_engine.retriever._similarity_top_k = 10

In [ ]:
response = query_engine.query("please change the Background to Blue")

In [ ]:
print(response)

## Chat with a prompt template (ToDo)

In [ ]:
custom_prompt = Prompt("""\
Given a conversation (between Human and Assistant) and a follow up message from Human, \
rewrite the message to be a standalone question that captures all relevant context \
from the conversation.

<Chat History>
{chat_history}

<Follow Up Message>
{question}

<Standalone question>
""")

# list of (human_message, ai_message) tuples
custom_chat_history = [
    (
        'Hello assistant, we are having a insightful discussion about Paul Graham today.',
        'Okay, sounds good.'
    )
]

query_engine = index.as_query_engine()
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=custom_prompt,
    chat_history=custom_chat_history,
    verbose=True
)

## print token usage

In [ ]:
print('Embedding Tokens: ', token_counter.total_embedding_token_count, '\n',
      'LLM Prompt Tokens: ', token_counter.prompt_llm_token_count, '\n',
      'LLM Completion Tokens: ', token_counter.completion_llm_token_count, '\n',
      'Total LLM Token Count: ', token_counter.total_llm_token_count)